# Importando os módulos necessários

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, MinMaxScaler
from joblib import load, dump
from ClasseUtilitarios import Utilitarios

# Importando o dataset para treino do modelo

In [2]:
df = pd.read_csv('bank-full.csv', sep=';') # importando o dataset
df.head()  # visualizando dataset

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# Análise exploratória
- Apenas um exemplo de análise, deve-se realizar até mais análise

In [3]:
colunas_categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']

In [4]:
for c in colunas_categoricas:
    print (c)
    print (df[c].unique())

job
['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital
['married' 'single' 'divorced']
education
['tertiary' 'secondary' 'unknown' 'primary']
default
['no' 'yes']
housing
['yes' 'no']
loan
['no' 'yes']
contact
['unknown' 'cellular' 'telephone']
month
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
poutcome
['unknown' 'failure' 'other' 'success']
y
['no' 'yes']


In [5]:
# Podemos ver no resultado das colunas categóricas acima que DEFAULT, HOUSING, LOAN, Y podem ser usadas com LabelEncoder
# transformando no-> 0 e yes-> 1
colunas_LabelEncode = ['default', 'housing', 'loan']
label_encoders = {}
for c in colunas_LabelEncode:
    label_encoders[c] = LabelEncoder().fit(df[c])

label_encoders

{'default': LabelEncoder(), 'housing': LabelEncoder(), 'loan': LabelEncoder()}

In [6]:
encoder_output = LabelEncoder().fit(df.y)  # esse é o encoder da nossa variável dependente

In [7]:
# Sobraram então das colunas categórigas as seguintes: job, marital, education, contact, month e poutcome
# Para essas, vamos tentar fazer LabelBinarizer, que utiliza técnica One-Hot-Encode
colunas_LabelBinarizers = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
label_binarizers = {}
for c in colunas_LabelBinarizers:
    label_binarizers[c] = LabelBinarizer().fit(df[c])

label_binarizers

{'job': LabelBinarizer(),
 'marital': LabelBinarizer(),
 'education': LabelBinarizer(),
 'contact': LabelBinarizer(),
 'month': LabelBinarizer(),
 'poutcome': LabelBinarizer()}

---
- Até aqui criamos 3 encoders:  
- O primeiro é o LabelEncoder que salva todos as variáveis que decidimos transformar como labelEncode.  
- Segundo é o LabelBinarizer que salva os parâmetros para as variáveis que decidimos transformar no tipo One-Hot-Encoder.  
- Terceiro é o encoder da saída, no nosso caso colocamos encoder_output.
---
- Agora vamos importar a classe Utilitarios e usar algumas funções que criamos aqui mas separamos em um novo arquivo para que possamos reaproveitá-lo em outros ambientes, como produção.

In [8]:
utilitario = Utilitarios()

criado


In [9]:
# Vamos separar aqui o dataset em variáveis dependentes e variáveis independentes
y = df.y    # ainda está como variável categórica
y = encoder_output.transform(y)  # transformando nossa saída categórica em saída numérica

x = df.drop(columns=['y'])   # retirando a variável dependente
x = utilitario.pipeline(x, label_encoders_=label_encoders, label_binarizers_=label_binarizers) # transformando o df

In [10]:
# Aqui separando o dataset em treino-teste para validação cruzada
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y, shuffle=True)
print (x_train.shape, x_test.shape, y_train.shape, y_test.shape, sep=', ')

(40689, 48), (4522, 48), (40689,), (4522,)


In [12]:
# Criando agora nosso scaler e "fitando-transformando" nossos dados de treino e apenas transformando os dados de teste
scaler = MinMaxScaler()   # criando objeto scaler
scaler.fit(x_train)   # fitando
x_train = scaler.transform(x_train)  # transformando treino
x_test = scaler.transform(x_test)   # transformando teste

# Criando um modelo RandomForest

In [13]:
# Criando agora nosso modelo ENSEMBLE
model = RandomForestClassifier()  # instanciando
model.fit(x_train, y_train) # treinando
y_pred = model.predict(x_test) # inferência

accuracy = accuracy_score(y_pred, y_test)
precision = precision_score(y_pred, y_test)
recall = recall_score(y_pred, y_test)
f1score = f1_score(y_pred, y_test)

print ('Accuracy: {:0.2f}%\nPrecision: {:0.2f}%\nRecall: {:0.2f}%\nF1-Score: {:0.2f}%\n'.format(
    100*accuracy, 100*precision, 100*recall, 100*f1score
))

Accuracy: 90.16%
Precision: 35.73%
Recall: 64.29%
F1-Score: 45.93%



---
- Depois que escolhemos o nosso melhor modelo, podemos salvá-lo junto com todos os seus parâmetros para que depois possa ser reutilizado em outros ambientes e códigos.

In [14]:
# Criando um dicionário e salvando utilizando joblib.dump()
parametros_modelo = {
    'description': 'Modelo RandomForestClassifier treinado utilizando dataset UCI-Bank',
    'model': model,
    'scaler': scaler,
    'label_binarizers': label_binarizers,
    'label_encoders': label_encoders,
    'lastUpdate': '29-05-2021',
    'columns': df.columns.values,
    'encoder_output': encoder_output
}
dump(parametros_modelo, 'ensemble.joblib')

['ensemble.joblib']